In [112]:
!pip install requests
import requests

Looking in indexes: https://mirrors.cloud.aliyuncs.com/pypi/simple
DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: pytorch-lightning 1.7.7 has a non-standard dependency specifier torch>=1.9.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


# 精准搜索

In [95]:
def search_github_repositories(query, language=None, sort='best_match', order='desc', created=None, pushed=None, per_page=10):
    """
    使用 GitHub Search API 搜索仓库。

    参数:
    - query (str): 搜索关键词。
    - language (str, optional): 仓库编程语言。
    - sort (str, optional): 排序方式（'stars', 'forks', 'updated'）。
    - order (str, optional): 排序顺序（'asc', 'desc'）。
    - created (str, optional): 创建时间范围（例如 '>=2024-01-01'）。
    - pushed (str, optional): 最近更新时间范围（例如 '>=2024-01-01'）。
    - per_page (int, optional): 每页显示的结果数量。

    返回:
    - dict: 解析后的搜索结果。
    """
    # 构建搜索 URL
    url = 'https://api.github.com/search/repositories'
    params = {'q': query}

    if language:
        params['q'] += f' +language:{language}'

    if sort != 'best_match':
        params['sort'] = sort

    if order:
        params['order'] = order

    if created:
        params['q'] += f'+created:{created}'

    #if pushed:
    #    params['q'] += f'+pushed:{pushed}'

    if per_page:
        params['per_page'] = per_page

    # 发送请求
    print(f"url: {url}, params:{params}")

    response = requests.get(url, params=params)
    #print(f"response: {response}")
    # 检查响应状态码
    if response.status_code != 200:
        raise Exception(f"Error fetching data: {response.status_code}")

    # 解析 JSON 响应
    results = response.json()
    #print(f"results: {results}")

    # 解析并返回结果
    parsed_results = {
        'total_count': results['total_count'],
        'items': [
            {
                'name': item['name'],
                'full_name': item['full_name'],
                'html_url': item['html_url'],
                'description': item['description'],
                'stargazers_count': item['stargazers_count'],
                'forks_count': item['forks_count'],
                'open_issues_count': item['open_issues_count'],
                'language': item['language'],
                'created_at': item['created_at'],
                'updated_at': item['updated_at'],
            }
            for item in results['items']
        ]
    }

    #print(f"result:{parsed_results}")
    return parsed_results


# 读取指定未知的readme

In [118]:
import base64
import requests

def get_readme_content(full_name):
    """
    获取指定 GitHub 仓库的 README.md 文件内容。

    参数:
    - full_name (str): owner(用户名或组织名)/repo(仓库名称)

    返回:
    - str: README.md 文件的内容，如果是二进制文件则返回解码后的字符串。
    """
    url = f"https://api.github.com/repos/{full_name}/readme"
    response = requests.get(url)
    try:
        if response.status_code != 200:
            raise Exception(f"Error fetching data: {response.status_code}, {response.text}")

        content_info = response.json()
        # README 的内容是以 base64 编码的
        encoded_content = content_info['content']
        # 解码得到原始的文本内容
        decoded_content = base64.b64decode(encoded_content).decode('utf-8')

        return decoded_content
    except Exception as e:
        # 这里可以记录错误信息，或者采取其他措施
        print(f"An error occurred: {e}")
        return None


if 0:
    # 示例调用
    try:
        readme_content = get_readme_content(full_name="OpenAutoCoder/Agentless")
        print(readme_content)
    except Exception as e:
        print(e)


# 演示

In [119]:
# 示例调用
search_results = search_github_repositories(
    query='Agentless',
    #language='Python',
    #sort='updated',
    order='desc',
    #per_page=100
)

for repo in search_results['items']:
    print(f"Repo Name: {repo['name']}, Stars: {repo['stargazers_count']}, Language: {repo['language']} ...")
    print(f"""readme: {get_readme_content(repo['full_name'])}""")

readme: ![ci](https://github.com/abbbi/virtnbdbackup/actions/workflows/ci-ubuntu-latest.yml/badge.svg)
[![package-build](https://github.com/abbbi/virtnbdbackup/actions/workflows/build.yml/badge.svg)](https://github.com/abbbi/virtnbdbackup/actions/workflows/build.yml)

# virtnbdbackup

Backup utility for `libvirt`, using the latest changed block tracking features.
Create online, thin provisioned full and incremental or differential backups
of your `kvm/qemu` virtual machines.

![Alt text](screenshot.jpg?raw=true "Title")

<!-- START doctoc generated TOC please keep comment here to allow auto update -->
<!-- DON'T EDIT THIS SECTION, INSTEAD RE-RUN doctoc TO UPDATE -->

- [About](#about)
- [Prerequisites](#prerequisites)
  - [Libvirt versions <= 7.6.0 (Debian Bullseye, Ubuntu 20.x)](#libvirt-versions--760-debian-bullseye-ubuntu-20x)
  - [RHEL/Centos Stream, Alma, Rocky Linux](#rhelcentos-stream-alma-rocky-linux)
    - [Version <= 8.5](#version--85)
    - [Version >= 8.6](#version--86)
  -